In [ ]:
import os

In [ ]:
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')


In [ ]:
import requests
headers = {"Authorization": f"token {GITHUB_TOKEN}"}

In [ ]:
import time
import math
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm


def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=10_000,
    rate_limit=5_000,
    issues_path=Path("."),
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    batch = []
    all_issues = []
    per_page = 100  ## 每页返回的 issue 的数量
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        # 使用 state=all 进行查询来获取 open 和 closed 的issue
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # 重置batch
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

In [ ]:
from datasets import load_dataset

In [ ]:
issues_dataset = load_dataset('json',data_files='datasets-issues.jsonl',split='train')

In [ ]:
issues_dataset

In [ ]:
sample = issues_dataset.shuffle(seed=666).select(range(10))

In [ ]:
sample

In [ ]:
for url,pr in zip(sample['html_url'],sample['pull_request']):
    print(f">> URL: {url}")
    print(f">> Pull request: {pr}\n")

In [ ]:
issues_dataset = issues_dataset.map(lambda x: {'is_pull-rerquest': False if x['pull_request'] is None else True})

In [ ]:
issues_dataset[0]['number']

In [ ]:
issue_number = 7933

In [ ]:
url = f'https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments'

In [ ]:
response2 = requests.get(url,headers=headers)

In [ ]:
def get_comments(issue_number):
    url = f'https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments'
    response = requests.get(url,headers=headers)
    return [r['body'] for r in response.json()]

In [ ]:
issues_with_comments_dataset = issues_dataset.map(lambda x: {'comments': get_comments(x['number'])})

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
issues_with_comments_dataset.push_to_hub('github-issues')


In [ ]:
remote_dataset = load_dataset('YiMeng-SYSU/github-issues',split='train')
remote_dataset